In [1]:
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 48.6 MB/s eta 0:00:00


In [80]:
import geopandas as gpd
import numpy as np
import pandas as pd
from shapely import Point, Polygon
import folium
from tqdm import tqdm
import warnings

warnings.filterwarnings('ignore')

In [3]:
schools_df = gpd.read_file('/content/schools_datamos_final.geojson')
kindergartens_df = gpd.read_file('/content/kindergartens_datamos_final.geojson')
medicine_df = gpd.read_file('/content/medicine_transform.geojson')
buildings_df = gpd.read_file('/content/buildings_transform.geojson')
districts_df = gpd.read_file('/content/administrative_borders_transform.geojson')
regions_df = gpd.read_file('/content/administrative_regions_transform.geojson')

In [ ]:
districts_list = list(districts_df['district_id'])

# Считаем количество школ для зданий

In [ ]:
buildings_df['free_schools'] = ''
buildings_df['over_schools'] = ''

In [38]:
def swap_points(points):
        for j in range(len(points)):
            points[j] = points[j][::-1]

        return points

In [39]:
def intersction(df_objects, polygons_df):
        df_objects['centroid'] = df_objects.geometry.centroid
        polygons_df['polygon'] = polygons_df.geometry
        objects_df = df_objects.set_geometry('centroid')

        return gpd.sjoin(objects_df, polygons_df)

In [61]:
m = folium.Map()

In [81]:
def get_buffer(m, object_d, radius, df_objects):
        centroid_latitude = object_d['centroid latitude']
        centroid_longitude = object_d['centroid longitude']

        df = pd.DataFrame(
            {
                'lat': [centroid_latitude],
                'lon': [centroid_longitude],
                'rad': [radius]
            }
        )
        #Отрисовка круга нужного радиуса на карте. Так сложно потому что земля вам не шарик, а хер пойми что
        df['geom'] = df.apply(lambda r: Point(r['lon'], r['lat']), axis=1)
        gdf = gpd.GeoDataFrame(df, geometry='geom', crs='epsg:4326')
        gdf_flat = gdf.to_crs('epsg:6347')
        gdf_flat['geom'] = gdf_flat.geometry.buffer(df.rad)
        gdf = gdf_flat.to_crs('epsg:4326')
        points = list(list(gdf['geom'])[0].exterior.coords)
        points = swap_points(points)
        polyline = folium.PolyLine(locations=points)

        polyline.add_to(m)

        #Датафрейм для пересечения
        frame_for_inter = gpd.GeoDataFrame()
        frame_for_inter['geometry'] = [Polygon(swap_points(points))]

        df_inter_buffer = intersction(df_objects, frame_for_inter)
        df_inter_buffer_free = df_inter_buffer.loc[df_inter_buffer['workload'] < 100]
        df_inter_buffer_over = df_inter_buffer.loc[df_inter_buffer['workload'] >= 100]

        return len(df_inter_buffer_free), len(df_inter_buffer_over)

In [ ]:
id_map = {}
districts_list = districts_list[::-1]
for i in tqdm(range(len(districts_list))):
  sub_df = buildings_df.loc[buildings_df['district_id'] == districts_list[i]]
  distrcit_id_list = ['relation/1257484', 'relation/2162195', 'relation/1255942', 'relation/1257218',
                              'relation/364001', 'relation/1275551', 'relation/1275608', 'relation/1257786',
                              'relation/1255987', 'relation/1275627']
  list_number = []
  district_id = list(sub_df.iloc[0]['district_id'])[0]
  if district_id in distrcit_id_list:
    radius = 750
  else:
    radius = 500
  for i in range(len(sub_df)):
    object_d = sub_df.iloc[i]
    id = sub_df.iloc[i]['id']
    number_of_free_schools_in_radius, number_of_over_schools_in_radius = get_buffer(m, object_d, radius, schools_df)
    #list_number.append([number_of_free_schools_in_radius, number_of_over_schools_in_radius, number_of_free_schools_in_radius + number_of_over_schools_in_radius])
    buildings_df.loc[buildings_df['id'] == id, 'free_schools'] = number_of_free_schools_in_radius
    buildings_df.loc[buildings_df['id'] == id, 'over_schools'] = number_of_over_schools_in_radius
    #id_map[id] = [number_of_free_schools_in_radius, number_of_over_schools_in_radius]

In [92]:
buildings_df.to_file('/content/buildings_transform.geojson', driver="GeoJSON")

In [ ]:
m

#Считаем количество детсадов для зданий

In [106]:
buildings_df['avaliable_kindergartens'] = ''

In [107]:
def swap_points(points):
        for j in range(len(points)):
            points[j] = points[j][::-1]

        return points

In [108]:
def intersction(df_objects, polygons_df):
        df_objects['centroid'] = df_objects.geometry.centroid
        polygons_df['polygon'] = polygons_df.geometry
        objects_df = df_objects.set_geometry('centroid')

        return gpd.sjoin(objects_df, polygons_df)

In [109]:
m = folium.Map()

In [110]:
def get_buffer_k(m, object_d, radius, df_objects):
        centroid_latitude = object_d['centroid latitude']
        centroid_longitude = object_d['centroid longitude']

        df = pd.DataFrame(
            {
                'lat': [centroid_latitude],
                'lon': [centroid_longitude],
                'rad': [radius]
            }
        )
        #Отрисовка круга нужного радиуса на карте. Так сложно потому что земля вам не шарик, а хер пойми что
        df['geom'] = df.apply(lambda r: Point(r['lon'], r['lat']), axis=1)
        gdf = gpd.GeoDataFrame(df, geometry='geom', crs='epsg:4326')
        gdf_flat = gdf.to_crs('epsg:6347')
        gdf_flat['geom'] = gdf_flat.geometry.buffer(df.rad)
        gdf = gdf_flat.to_crs('epsg:4326')
        points = list(list(gdf['geom'])[0].exterior.coords)
        points = swap_points(points)
        polyline = folium.PolyLine(locations=points)

        polyline.add_to(m)

        #Датафрейм для пересечения
        frame_for_inter = gpd.GeoDataFrame()
        frame_for_inter['geometry'] = [Polygon(swap_points(points))]

        df_inter_buffer = intersction(df_objects, frame_for_inter)

        return len(df_inter_buffer)

In [114]:
for i in tqdm(range(len(districts_list))):
  sub_df = buildings_df.loc[buildings_df['district_id'] == districts_list[i]]
  distrcit_id_list = ['relation/1257484', 'relation/2162195', 'relation/1255942', 'relation/1257218',
                              'relation/364001', 'relation/1275551', 'relation/1275608', 'relation/1257786',
                              'relation/1255987', 'relation/1275627']
  list_number = []
  district_id = i
  if district_id in distrcit_id_list:
    radius = 500
  else:
    radius = 300
  for i in range(len(sub_df)):
    object_d = sub_df.iloc[i]
    id = sub_df.iloc[i]['id']
    number_of_kindergartens_in_radius = get_buffer_k(m, object_d, radius, kindergartens_df)
    buildings_df.loc[buildings_df['id'] == id, 'avaliable_kindergartens'] = number_of_kindergartens_in_radius

100%|██████████| 145/145 [1:31:33<00:00, 37.88s/it]


In [115]:
buildings_df.to_file('/content/buildings_transform.geojson', driver="GeoJSON")

In [116]:
buildings_df

,id,addr:city,addr:housenumber,addr:postcode,addr:street,building,building:flats,building:levels,design:ref,centroid latitude,...,year,region_id,region_name,kindergartens,Pupils,adults,geometry,free_schools,over_schools,avaliable_kindergartens
0,relation/58174,Москва,6,000000,улица Довженко,apartments,912,10,1-515/9М,55.722578,...,1976,relation/226149,Западный административный округ,99,258,1537,"POLYGON ((37.51454 55.72195, 37.51475 55.72208...",0.0,1.0,1
1,relation/1822659,Москва,1 кБ,000000,empty,dormitory,0,18,empty,55.702349,...,NaN,relation/226149,Западный административный округ,137,358,2129,"POLYGON ((37.53149 55.70189, 37.53160 55.70195...",0.0,0.0,0
2,relation/1822660,Москва,1 кВ,000000,empty,dormitory,0,18,empty,55.703472,...,NaN,relation/226149,Западный административный округ,135,352,2094,"POLYGON ((37.53004 55.70388, 37.53017 55.70381...",0.0,0.0,0
3,relation/1822661,Москва,1 кГ,000000,empty,dormitory,0,9,empty,55.701592,...,NaN,relation/226149,Западный административный округ,31,82,485,"POLYGON ((37.53160 55.70195, 37.53149 55.70189...",0.0,0.0,0
4,relation/1822662,Москва,1 кД,000000,empty,dormitory,0,9,empty,55.703482,...,NaN,relation/226149,Западный административный округ,31,82,486,"POLYGON ((37.52896 55.70328, 37.52903 55.70332...",0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33999,way/816879973,Москва,4 к2,111677,улица Вертолётчиков,apartments,256,17,empty,55.704216,...,NaN,relation/1278703,Юго-Восточный административный округ,48,124,740,"POLYGON ((37.94449 55.70461, 37.94378 55.70379...",0.0,1.0,0
34000,way/821336533,Москва,12 к5,000000,улица Маресьева,apartments,0,14,empty,55.713050,...,NaN,relation/1278703,Юго-Восточный административный округ,38,100,593,"POLYGON ((37.94334 55.71300, 37.94407 55.71280...",0.0,0.0,0
34001,way/844852891,Москва,6 к1,111677,улица Маресьева,apartments,231,22,empty,55.709866,...,NaN,relation/1278703,Юго-Восточный административный округ,36,95,563,"POLYGON ((37.93633 55.71006, 37.93579 55.70972...",0.0,0.0,0
34002,way/916147076,Москва,10 к2,000000,улица Маресьева,apartments,0,17,empty,55.712484,...,NaN,relation/1278703,Юго-Восточный административный округ,40,103,616,"POLYGON ((37.94094 55.71267, 37.94078 55.71276...",0.0,0.0,0


#Доступность для мед

In [120]:
buildings_df['avaliable_medicine'] = ''

In [121]:
def swap_points(points):
        for j in range(len(points)):
            points[j] = points[j][::-1]

        return points

In [122]:
def intersction(df_objects, polygons_df):
        df_objects['centroid'] = df_objects.geometry.centroid
        polygons_df['polygon'] = polygons_df.geometry
        objects_df = df_objects.set_geometry('centroid')

        return gpd.sjoin(objects_df, polygons_df)

In [123]:
def get_buffer_m(m, object_d, radius, df_objects):
        centroid_latitude = object_d['centroid latitude']
        centroid_longitude = object_d['centroid longitude']

        df = pd.DataFrame(
            {
                'lat': [centroid_latitude],
                'lon': [centroid_longitude],
                'rad': [radius]
            }
        )
        #Отрисовка круга нужного радиуса на карте. Так сложно потому что земля вам не шарик, а хер пойми что
        df['geom'] = df.apply(lambda r: Point(r['lon'], r['lat']), axis=1)
        gdf = gpd.GeoDataFrame(df, geometry='geom', crs='epsg:4326')
        gdf_flat = gdf.to_crs('epsg:6347')
        gdf_flat['geom'] = gdf_flat.geometry.buffer(df.rad)
        gdf = gdf_flat.to_crs('epsg:4326')
        points = list(list(gdf['geom'])[0].exterior.coords)
        points = swap_points(points)
        polyline = folium.PolyLine(locations=points)

        polyline.add_to(m)

        #Датафрейм для пересечения
        frame_for_inter = gpd.GeoDataFrame()
        frame_for_inter['geometry'] = [Polygon(swap_points(points))]

        df_inter_buffer = intersction(df_objects, frame_for_inter)

        return len(df_inter_buffer)

In [124]:
for i in tqdm(range(len(districts_list))):
  sub_df = buildings_df.loc[buildings_df['district_id'] == districts_list[i]]
  distrcit_id_list = ['relation/1257484', 'relation/2162195', 'relation/1255942', 'relation/1257218',
                              'relation/364001', 'relation/1275551', 'relation/1275608', 'relation/1257786',
                              'relation/1255987', 'relation/1275627']
  list_number = []
  #district_id = i
  #if district_id in distrcit_id_list:
  #  radius = 500
  #else:
  #  radius = 300
  radius = 1500
  for i in range(len(sub_df)):
    object_d = sub_df.iloc[i]
    id = sub_df.iloc[i]['id']
    number_of_medicine_in_radius = get_buffer_m(m, object_d, radius, medicine_df)
    buildings_df.loc[buildings_df['id'] == id, 'avaliable_medicine'] = number_of_medicine_in_radius

100%|██████████| 145/145 [1:31:36<00:00, 37.91s/it]


In [125]:
buildings_df.to_file('/content/buildings_transform.geojson', driver="GeoJSON")

In [126]:
buildings_df

,id,addr:city,addr:housenumber,addr:postcode,addr:street,building,building:flats,building:levels,design:ref,centroid latitude,...,region_id,region_name,kindergartens,Pupils,adults,geometry,free_schools,over_schools,avaliable_kindergartens,avaliable_medicine
0,relation/58174,Москва,6,000000,улица Довженко,apartments,912,10,1-515/9М,55.722578,...,relation/226149,Западный административный округ,99,258,1537,"POLYGON ((37.51454 55.72195, 37.51475 55.72208...",0.0,1.0,1,6
1,relation/1822659,Москва,1 кБ,000000,empty,dormitory,0,18,empty,55.702349,...,relation/226149,Западный административный округ,137,358,2129,"POLYGON ((37.53149 55.70189, 37.53160 55.70195...",0.0,0.0,0,11
2,relation/1822660,Москва,1 кВ,000000,empty,dormitory,0,18,empty,55.703472,...,relation/226149,Западный административный округ,135,352,2094,"POLYGON ((37.53004 55.70388, 37.53017 55.70381...",0.0,0.0,0,8
3,relation/1822661,Москва,1 кГ,000000,empty,dormitory,0,9,empty,55.701592,...,relation/226149,Западный административный округ,31,82,485,"POLYGON ((37.53160 55.70195, 37.53149 55.70189...",0.0,0.0,0,12
4,relation/1822662,Москва,1 кД,000000,empty,dormitory,0,9,empty,55.703482,...,relation/226149,Западный административный округ,31,82,486,"POLYGON ((37.52896 55.70328, 37.52903 55.70332...",0.0,0.0,0,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33999,way/816879973,Москва,4 к2,111677,улица Вертолётчиков,apartments,256,17,empty,55.704216,...,relation/1278703,Юго-Восточный административный округ,48,124,740,"POLYGON ((37.94449 55.70461, 37.94378 55.70379...",0.0,1.0,0,4
34000,way/821336533,Москва,12 к5,000000,улица Маресьева,apartments,0,14,empty,55.713050,...,relation/1278703,Юго-Восточный административный округ,38,100,593,"POLYGON ((37.94334 55.71300, 37.94407 55.71280...",0.0,0.0,0,0
34001,way/844852891,Москва,6 к1,111677,улица Маресьева,apartments,231,22,empty,55.709866,...,relation/1278703,Юго-Восточный административный округ,36,95,563,"POLYGON ((37.93633 55.71006, 37.93579 55.70972...",0.0,0.0,0,2
34002,way/916147076,Москва,10 к2,000000,улица Маресьева,apartments,0,17,empty,55.712484,...,relation/1278703,Юго-Восточный административный округ,40,103,616,"POLYGON ((37.94094 55.71267, 37.94078 55.71276...",0.0,0.0,0,0


#Статистика по районам

In [129]:
districts_df['schools_number'] = ''
districts_df['schools_workload'] = ''
districts_df['kindergartens_number'] = ''
districts_df['medicine_number'] = ''
districts_df['buildings_number'] = ''
districts_df['residents_number'] = ''
districts_df['avg_year'] = ''
districts_df['without_schools'] = ''
districts_df['without_kindergartens'] = ''
districts_df['without_medicine'] = ''

In [179]:
for i in range(len(districts_list)):
  #Количество школ
  sub_df = schools_df.loc[schools_df['district_id'] == districts_list[i]]
  school_count = sub_df.shape[0]
  #Средняя загруженность школ
  avg_workload = np.mean(list(sub_df['workload']))
  #Количество детсадов
  sub_df = kindergartens_df.loc[kindergartens_df['district_id'] == districts_list[i]]
  kindergartens_count = sub_df.shape[0]
  #Количество медицины
  sub_df = medicine_df.loc[medicine_df['district_id'] == districts_list[i]]
  medicin_count = sub_df.shape[0]
  #Количество жилых зданий
  sub_df = buildings_df.loc[buildings_df['district_id'] == districts_list[i]]
  buildings_count = sub_df.shape[0]
  #Количество жителей
  cols_list = ['kindergartens', 'Pupils', 'adults']
  total_number_of_people = sub_df[cols_list].astype(int).sum()
  total_number_of_people = total_number_of_people[0] + total_number_of_people[1] + total_number_of_people[2]
  buildings_residents = total_number_of_people
  #Средний год постройки зданий
  try:
    year_list = list(sub_df['year'].replace('н.д.', np.nan).dropna().astype(int))
    buildings_avg_year = int(sum(year_list) / len(year_list))
  except ZeroDivisionError:
    buildings_avg_year = -1
  #Процент зданий, которые находятся вне зоны пешей доступности для школ
  try:
    counter_over = 0
    for j in range(len(sub_df)):
      fs = sub_df.iloc[j]['free_schools']
      os = sub_df.iloc[j]['over_schools']
      if fs + os == 0:
        counter_over += 1
    percent_of_over_school = int(counter_over / len(sub_df) * 100)
  except ZeroDivisionError:
    percent_of_over_school = -1
  #Процент зданий, которые находятся вне зоны пешей доступности для детских садов
  try:
    counter_over = 0
    for j in range(len(sub_df)):
      ka = sub_df.iloc[j]['avaliable_kindergartens']
      if ka == 0:
        counter_over += 1
    percent_of_over_kinder = int(counter_over / len(sub_df) * 100)
  except ZeroDivisionError:
    percent_of_over_kinder = -1
  #Процент зданий, которые находятся вне зоны пешей доступности для медицинских учреждений
  try:
    counter_over = 0
    for j in range(len(sub_df)):
      ma = sub_df.iloc[j]['avaliable_medicine']
      if ma == 0:
        counter_over += 1
    percent_of_over_medicine = int(counter_over / len(sub_df) * 100)
  except ZeroDivisionError:
    percent_of_over_medicine = -1

  districts_df.loc[districts_df['district_id'] == districts_list[i], 'schools_number'] = str(school_count)
  districts_df.loc[districts_df['district_id'] == districts_list[i], 'schools_workload'] = str(avg_workload)
  districts_df.loc[districts_df['district_id'] == districts_list[i], 'kindergartens_number'] = str(kindergartens_count)
  districts_df.loc[districts_df['district_id'] == districts_list[i], 'medicine_number'] = str(medicin_count)
  districts_df.loc[districts_df['district_id'] == districts_list[i], 'buildings_number'] = str(buildings_count)
  districts_df.loc[districts_df['district_id'] == districts_list[i], 'residents_number'] = str(buildings_residents)
  districts_df.loc[districts_df['district_id'] == districts_list[i], 'avg_year'] = str(buildings_avg_year)
  districts_df.loc[districts_df['district_id'] == districts_list[i], 'without_schools'] = str(percent_of_over_school)
  districts_df.loc[districts_df['district_id'] == districts_list[i], 'without_kindergartens'] = str(percent_of_over_kinder)
  districts_df.loc[districts_df['district_id'] == districts_list[i], 'without_medicine'] = str(percent_of_over_medicine)

In [180]:
districts_df

,district_id,addr:region,boundary,district_name,district_area,geometry,schools_number,schools_workload,kindergartens_number,medicine_number,buildings_number,residents_number,avg_year,without_schools,without_kindergartens,without_medicine
0,relation/181288,Москва,administrative,район Внуково,1.542062e+07,"MULTIPOLYGON (((37.19085 55.60184, 37.19088 55...",2,296.0,5,6,79,15059,1970,48,70,1
1,relation/226927,Москва,administrative,район Кунцево,5.265320e+07,"MULTIPOLYGON (((37.37864 55.80837, 37.37939 55...",24,149.25,25,28,395,163464,1972,18,48,0
2,relation/240229,Москва,administrative,район Крылатское,1.218994e+07,"POLYGON ((37.42256 55.74518, 37.42346 55.74587...",10,163.6,15,16,153,99617,1991,44,39,15
3,relation/364001,Москва,administrative,Мещанский район,4.515232e+06,"POLYGON ((37.62144 55.75929, 37.62142 55.75924...",12,137.5,11,56,293,69602,1951,30,57,0
4,relation/364551,Москва,administrative,район Сокольники,1.024107e+07,"POLYGON ((37.66909 55.82134, 37.66830 55.82018...",24,100.54166666666667,16,24,215,62888,1962,9,63,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,relation/2092927,Москва,administrative,поселение Михайлово-Ярцевское,6.469836e+07,"POLYGON ((37.10850 55.43951, 37.10650 55.43852...",1,145.0,0,1,30,3430,-1,53,100,46
141,relation/2092928,Москва,administrative,поселение Роговское,1.780844e+08,"POLYGON ((36.98644 55.31559, 36.98654 55.31465...",1,189.0,1,1,20,5944,-1,30,40,0
142,relation/2092929,Москва,administrative,поселение Рязановское,4.064429e+07,"POLYGON ((37.45463 55.49408, 37.45456 55.49405...",3,261.0,7,6,90,36504,1990,67,68,16
143,relation/2092931,Москва,administrative,поселение Щаповское,8.631793e+07,"POLYGON ((37.46539 55.39592, 37.46521 55.39659...",1,145.0,2,3,48,8615,-1,62,75,27


In [181]:
districts_df.to_file('/content/administrative_borders_transform.geojson', driver="GeoJSON")

#Статистика для округов

In [172]:
regions_df['schools_number'] = ''
regions_df['schools_workload'] = ''
regions_df['kindergartens_number'] = ''
regions_df['medicine_number'] = ''
regions_df['buildings_number'] = ''
regions_df['residents_number'] = ''
regions_df['avg_year'] = ''
regions_df['without_schools'] = ''
regions_df['without_kindergartens'] = ''
regions_df['without_medicine'] = ''

In [173]:
regions_list = list(regions_df['region_id'])

In [182]:
for i in range(len(regions_list)):
  #Количество школ
  sub_df = schools_df.loc[schools_df['region_id'] == regions_list[i]]
  school_count = sub_df.shape[0]
  #Средняя загруженность школ
  avg_workload = np.mean(list(sub_df['workload']))
  #Количество детсадов
  sub_df = kindergartens_df.loc[kindergartens_df['region_id'] == regions_list[i]]
  kindergartens_count = sub_df.shape[0]
  #Количество медицины
  sub_df = medicine_df.loc[medicine_df['region_id'] == regions_list[i]]
  medicin_count = sub_df.shape[0]
  #Количество жилых зданий
  sub_df = buildings_df.loc[buildings_df['region_id'] == regions_list[i]]
  buildings_count = sub_df.shape[0]
  #Количество жителей
  cols_list = ['kindergartens', 'Pupils', 'adults']
  total_number_of_people = sub_df[cols_list].astype(int).sum()
  total_number_of_people = total_number_of_people[0] + total_number_of_people[1] + total_number_of_people[2]
  buildings_residents = total_number_of_people
  #Средний год постройки зданий
  try:
    year_list = list(sub_df['year'].replace('н.д.', np.nan).dropna().astype(int))
    buildings_avg_year = int(sum(year_list) / len(year_list))
  except ZeroDivisionError:
    buildings_avg_year = -1
  #Процент зданий, которые находятся вне зоны пешей доступности для школ
  try:
    counter_over = 0
    for j in range(len(sub_df)):
      fs = sub_df.iloc[j]['free_schools']
      os = sub_df.iloc[j]['over_schools']
      if fs + os == 0:
        counter_over += 1
    percent_of_over_school = int(counter_over / len(sub_df) * 100)
  except ZeroDivisionError:
    percent_of_over_school = -1
  #Процент зданий, которые находятся вне зоны пешей доступности для детских садов
  try:
    counter_over = 0
    for j in range(len(sub_df)):
      ka = sub_df.iloc[j]['avaliable_kindergartens']
      if ka == 0:
        counter_over += 1
    percent_of_over_kinder = int(counter_over / len(sub_df) * 100)
  except ZeroDivisionError:
    percent_of_over_kinder = -1
  #Процент зданий, которые находятся вне зоны пешей доступности для медицинских учреждений
  try:
    counter_over = 0
    for j in range(len(sub_df)):
      ma = sub_df.iloc[j]['avaliable_medicine']
      if ma == 0:
        counter_over += 1
    percent_of_over_medicine = int(counter_over / len(sub_df) * 100)
  except ZeroDivisionError:
    percent_of_over_medicine = -1

  regions_df.loc[regions_df['region_id'] == regions_list[i], 'schools_number'] = str(school_count)
  regions_df.loc[regions_df['region_id'] == regions_list[i], 'schools_workload'] = str(avg_workload)
  regions_df.loc[regions_df['region_id'] == regions_list[i], 'kindergartens_number'] = str(kindergartens_count)
  regions_df.loc[regions_df['region_id'] == regions_list[i], 'medicine_number'] = str(medicin_count)
  regions_df.loc[regions_df['region_id'] == regions_list[i], 'buildings_number'] = str(buildings_count)
  regions_df.loc[regions_df['region_id'] == regions_list[i], 'residents_number'] = str(buildings_residents)
  regions_df.loc[regions_df['region_id'] == regions_list[i], 'avg_year'] = str(buildings_avg_year)
  regions_df.loc[regions_df['region_id'] == regions_list[i], 'without_schools'] = str(percent_of_over_school)
  regions_df.loc[regions_df['region_id'] == regions_list[i], 'without_kindergartens'] = str(percent_of_over_kinder)
  regions_df.loc[regions_df['region_id'] == regions_list[i], 'without_medicine'] = str(percent_of_over_medicine)

In [183]:
regions_df

,region_id,addr:region,boundary,region_name,region_area,geometry,schools_number,schools_workload,kindergartens_number,medicine_number,buildings_number,residents_number,avg_year,without_schools,without_kindergartens,without_medicine
0,relation/162903,Москва,administrative,Северный административный округ,1.023516e+08,"MULTIPOLYGON (((37.39002 55.91284, 37.39426 55...",187,136.13368983957218,236,268,3742,1357471,1964,12,37,0
1,relation/226149,Москва,administrative,Западный административный округ,1.939465e+08,"MULTIPOLYGON (((36.91270 55.74751, 36.90939 55...",182,150.7912087912088,221,255,3297,1558717,1975,25,49,4
2,relation/446092,Москва,administrative,Северо-Западный административный округ,9.325469e+07,"POLYGON ((37.38505 55.80931, 37.38496 55.80906...",138,156.7246376811594,185,196,2387,1014638,1976,18,39,0
3,relation/1252558,Москва,administrative,Северо-Восточный административный округ,1.016799e+08,"POLYGON ((37.58811 55.79259, 37.58822 55.79257...",201,142.51243781094527,238,215,3369,1335424,1974,15,40,0
4,relation/1278703,Москва,administrative,Юго-Восточный административный округ,1.228987e+08,"MULTIPOLYGON (((37.96522 55.70970, 37.96597 55...",223,146.33183856502242,254,202,3589,1447903,1975,13,34,1
5,relation/1282181,Москва,administrative,Южный административный округ,1.315780e+08,"POLYGON ((37.66428 55.57163, 37.66637 55.57167...",235,142.68085106382978,295,226,3328,1508858,1974,14,33,0
6,relation/1304596,Москва,administrative,Юго-Западный административный округ,1.113623e+08,"POLYGON ((37.58930 55.55720, 37.58945 55.55768...",233,144.11158798283262,294,212,3100,1505897,1974,12,34,0
7,relation/1320234,Москва,administrative,Восточный административный округ,1.549069e+08,"MULTIPOLYGON (((37.70074 55.86092, 37.70269 55...",232,144.0905172413793,284,265,4431,1380407,1969,10,33,0
8,relation/2162196,Москва,administrative,Центральный административный округ,6.620456e+07,"POLYGON ((37.58780 55.79264, 37.58823 55.79236...",187,123.475935828877,156,649,3896,1094674,1926,10,49,0
9,relation/2263058,Москва,administrative,Новомосковский административный округ,3.597162e+08,"POLYGON ((37.26095 55.64961, 37.26048 55.64962...",37,217.35135135135135,67,74,1912,950832,1999,62,73,18


In [184]:
regions_df.to_file('/content/administrative_regions_transform.geojson', driver="GeoJSON")